In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, RandomUnderSampler

from dsto_gan import DSTO_GAN

In [39]:
# Carregar dataset Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Selecionar atributos relevantes
df = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
df.dropna(inplace=True)  # Remover linhas com valores nulos

# Transformar variáveis categóricas em numéricas
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Embarked'] = le.fit_transform(df['Embarked'])

# Separar features e target
X = df.drop('Survived', axis=1)
y = df['Survived']

# Dividir os dados em treino e teste (mantendo desbalanceamento original)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


In [41]:
tomek = TomekLinks()
X_train_tomek, y_train_tomek = tomek.fit_resample(X_train, y_train)


In [42]:
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)


In [43]:
sto_gan = DSTO_GAN()

# Convert DataFrame to NumPy array
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)

X_train_dsto, y_train_dsto = dsto_gan.fit_resample(X_train_np, y_train_np)


In [44]:
# Convert NumPy array back to DataFrame
X_train_dsto = pd.DataFrame(X_train_dsto, columns=X_train.columns)

# Now fit the model
rf_model.fit(X_train_dsto, y_train_dsto)

RandomForestClassifier(random_state=42)

In [45]:
# Definir a função de avaliação
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return {
        'Acurácia': accuracy_score(y_test, y_pred),
        'Precisão': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }

# Escolher modelo base (Random Forest)
rf_model = RandomForestClassifier(random_state=42)

# Avaliar os diferentes métodos de balanceamento
results = {
    "Original": evaluate_model(rf_model, X_train, y_train, X_test, y_test),
    "SMOTE": evaluate_model(rf_model, X_train_smote, y_train_smote, X_test, y_test),
    "Tomek Links": evaluate_model(rf_model, X_train_tomek, y_train_tomek, X_test, y_test),
    "Random Undersampling": evaluate_model(rf_model, X_train_rus, y_train_rus, X_test, y_test),
    "DSTO-GAN": evaluate_model(rf_model, X_train_dsto, y_train_dsto, X_test, y_test)
}

# Exibir os resultados
for method, metrics in results.items():
    print(f"\n{method}:\n", metrics)



Original:
 {'Acurácia': 0.7902097902097902, 'Precisão': 0.7796610169491526, 'Recall': 0.7301587301587301, 'F1-Score': 0.7540983606557377}

SMOTE:
 {'Acurácia': 0.7832167832167832, 'Precisão': 0.7580645161290323, 'Recall': 0.746031746031746, 'F1-Score': 0.752}

Tomek Links:
 {'Acurácia': 0.7622377622377622, 'Precisão': 0.7230769230769231, 'Recall': 0.746031746031746, 'F1-Score': 0.734375}

Random Undersampling:
 {'Acurácia': 0.7692307692307693, 'Precisão': 0.7205882352941176, 'Recall': 0.7777777777777778, 'F1-Score': 0.7480916030534351}

DSTO-GAN:
 {'Acurácia': 0.7762237762237763, 'Precisão': 0.7540983606557377, 'Recall': 0.7301587301587301, 'F1-Score': 0.7419354838709677}
